# Введение
Линейные методы хорошо подходят для работы с разреженными данными — к таковым относятся, например, тексты. Это можно объяснить высокой скоростью обучения и небольшим количеством параметров, благодаря чему удается избежать переобучения.

Линейная регрессия имеет несколько разновидностей в зависимости от того, какой регуляризатор используется. Мы будем работать с гребневой регрессией, где применяется квадратичный, или L2-регуляризатор.

Для извлечения TF-IDF-признаков из текстов воспользуйтесь классом sklearn.feature_extraction.text.TfidfVectorizer.

Для предсказания целевой переменной мы будем использовать гребневую регрессию, которая реализована в классе sklearn.linear_model.Ridge.

Обратите внимание, что признаки LocationNormalized и ContractTime являются строковыми, и поэтому с ними нельзя работать напрямую. Такие нечисловые признаки с неупорядоченными значениями называют категориальными или номинальными. Типичный подход к их обработке — кодирование категориального признака с m возможными значениями с помощью m бинарных признаков. Каждый бинарный признак соответствует одному из возможных значений категориального признака и является индикатором того, что на данном объекте он принимает данное значение. Данный подход иногда называют one-hot-кодированием. Воспользуйтесь им, чтобы перекодировать признаки LocationNormalized и ContractTime. Он уже реализован в классе sklearn.feature_extraction.DictVectorizer.

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge

In [2]:
salary_train = pd.read_csv('_df0abf627c1cd98b7332b285875e7fe9_salary-train.csv')
salary_train

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355
...,...,...,...,...
59995,"As a result of continued growth, First Class S...",Whitley Bay,contract,26400
59996,PHP / MVC Web Developer MacclesfieldCirca ***...,Macclesfield,permanent,26000
59997,"Staff Nurse, Nursing Home, Baldock White Recru...",Baldock,NaN,24500
59998,This is one of the best agency side opportunit...,The City,permanent,65000


Вам понадобится производить замену пропущенных значений на специальные строковые величины (например, 'nan').

In [3]:
salary_train['LocationNormalized'].fillna('nan', inplace=True)
salary_train['ContractTime'].fillna('nan', inplace=True)
salary_train

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,nan,20355
...,...,...,...,...
59995,"As a result of continued growth, First Class S...",Whitley Bay,contract,26400
59996,PHP / MVC Web Developer MacclesfieldCirca ***...,Macclesfield,permanent,26000
59997,"Staff Nurse, Nursing Home, Baldock White Recru...",Baldock,nan,24500
59998,This is one of the best agency side opportunit...,The City,permanent,65000


In [4]:
salary_test = pd.read_csv('_d0f655638f1d87a0bdeb3bad26099ecd_salary-test-mini.csv')
salary_test

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [5]:
y_train = pd.Series(salary_train.pop('SalaryNormalized')).to_frame()
y_train

,SalaryNormalized
0,33000
1,50000
2,40000
3,22500
4,20355
...,...
59995,26400
59996,26000
59997,24500
59998,65000


Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

In [6]:
enc = DictVectorizer()
X_train = enc.fit_transform(salary_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_train

<60000x1766 sparse matrix of type '<class 'numpy.float64'>'
	with 120000 stored elements in Compressed Sparse Row format>

In [7]:
X_test = enc.transform(salary_test[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test

<2x1766 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

Приведите тексты к нижнему регистру

In [8]:
salary_train['FullDescription'] = salary_train['FullDescription'].str.lower()
salary_test['FullDescription'] = salary_test['FullDescription'].str.lower()
salary_train

,FullDescription,LocationNormalized,ContractTime
0,international sales manager london ****k ****...,London,permanent
1,an ideal opportunity for an individual that ha...,London,permanent
2,online content and brand manager// luxury reta...,South East London,permanent
3,a great local marketleader is seeking a perman...,Dereham,permanent
4,registered nurse / rgn nursing home for young...,Sutton Coldfield,nan
...,...,...,...
59995,"as a result of continued growth, first class s...",Whitley Bay,contract
59996,php / mvc web developer macclesfieldcirca ***...,Macclesfield,permanent
59997,"staff nurse, nursing home, baldock white recru...",Baldock,nan
59998,this is one of the best agency side opportunit...,The City,permanent


In [9]:
salary_test

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,Milton Keynes,contract,NaN
1,a web developer opportunity has arisen with an...,Manchester,permanent,NaN


Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты:

In [10]:
salary_train['FullDescription'] = salary_train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
salary_test['FullDescription'] = salary_test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
salary_train

,FullDescription,LocationNormalized,ContractTime
0,international sales manager london k ...,London,permanent
1,an ideal opportunity for an individual that ha...,London,permanent
2,online content and brand manager luxury reta...,South East London,permanent
3,a great local marketleader is seeking a perman...,Dereham,permanent
4,registered nurse rgn nursing home for young...,Sutton Coldfield,nan
...,...,...,...
59995,as a result of continued growth first class s...,Whitley Bay,contract
59996,php mvc web developer macclesfieldcirca ...,Macclesfield,permanent
59997,staff nurse nursing home baldock white recru...,Baldock,nan
59998,this is one of the best agency side opportunit...,The City,permanent


In [11]:
salary_test

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,Milton Keynes,contract,NaN
1,a web developer opportunity has arisen with an...,Manchester,permanent,NaN


Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

In [12]:
vectorizer = TfidfVectorizer(min_df=5)
X_train_text_vector = vectorizer.fit_transform(salary_train['FullDescription'])
X_train_text_vector

<60000x22861 sparse matrix of type '<class 'numpy.float64'>'
	with 8365759 stored elements in Compressed Sparse Row format>

In [13]:
X_test_text_vector = vectorizer.transform(salary_test['FullDescription'])
X_test_text_vector

<2x22861 sparse matrix of type '<class 'numpy.float64'>'
	with 300 stored elements in Compressed Sparse Row format>

Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

In [14]:
from scipy.sparse import hstack

In [15]:
X_train = hstack([X_train, X_train_text_vector])
X_train

<60000x24627 sparse matrix of type '<class 'numpy.float64'>'
	with 8485759 stored elements in COOrdinate format>

In [16]:
X_test = hstack([X_test, X_test_text_vector])
X_test

<2x24627 sparse matrix of type '<class 'numpy.float64'>'
	with 304 stored elements in COOrdinate format>

In [17]:
ridge_regression = Ridge(alpha=1, random_state=241)

In [18]:
ridge_regression.fit(X_train, y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=241, solver='auto', tol=0.001)

Постройте прогнозы для двух примеров из файла salary-test-mini.csv.

In [19]:
ridge_regression.predict(X_test)

array([[56562.67153853],
       [37144.21466197]])